In [3]:
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from model import DAE_KAN_Attention
from histopathology_dataset import *
from torch.utils.data import Dataset, DataLoader
import wandb

In [4]:

train_ds = ImageDataset(*create_dataset('train'), 'cuda')
test_ds = ImageDataset(*create_dataset('test'), 'cuda')

train_dl = DataLoader(train_ds, batch_size=12, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=12, shuffle=True)


FileNotFoundError: [Errno 2] No such file or directory: './data/processed/train.csv'

In [2]:
model = DAE_KAN_Attention()
# checkpoint = torch.load("histo-dae/go2j1m2y/checkpoints/epoch=3-step=9528.ckpt")
checkpoint = torch.load("histopathology/histo-dae/d8mmjonm/checkpoints/epoch=29-step=24415.ckpt")


In [3]:
checkpoint.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])

In [4]:
checkpoint['state_dict'].keys()

odict_keys(['model.ae_encoder.kan.convs.0.conv.base_weight', 'model.ae_encoder.kan.convs.0.conv.spline_weight', 'model.ae_encoder.kan.convs.0.conv.spline_scaler', 'model.ae_encoder.kan.convs.0.conv.grid', 'model.ae_encoder.encoder1.0.weight', 'model.ae_encoder.encoder1.0.bias', 'model.ae_encoder.encoder1.1.weight', 'model.ae_encoder.encoder1.1.bias', 'model.ae_encoder.encoder1.1.running_mean', 'model.ae_encoder.encoder1.1.running_var', 'model.ae_encoder.encoder1.1.num_batches_tracked', 'model.ae_encoder.encoder2.0.weight', 'model.ae_encoder.encoder2.0.bias', 'model.ae_encoder.encoder2.1.weight', 'model.ae_encoder.encoder2.1.bias', 'model.ae_encoder.encoder2.1.running_mean', 'model.ae_encoder.encoder2.1.running_var', 'model.ae_encoder.encoder2.1.num_batches_tracked', 'model.ae_encoder.encoder3.0.weight', 'model.ae_encoder.encoder3.0.bias', 'model.ae_encoder.encoder3.1.weight', 'model.ae_encoder.encoder3.1.bias', 'model.ae_encoder.encoder3.1.running_mean', 'model.ae_encoder.encoder3.1.ru

In [5]:
new_state_dict = {}
for key, value in checkpoint['state_dict'].items():
    new_key = key.replace("model.", "")
    new_state_dict[new_key] = value

# Load the modified state dict into the model
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [6]:
from pprint import pprint as pp

pp(checkpoint['state_dict'].keys())

odict_keys(['model.ae_encoder.kan.convs.0.conv.base_weight', 'model.ae_encoder.kan.convs.0.conv.spline_weight', 'model.ae_encoder.kan.convs.0.conv.spline_scaler', 'model.ae_encoder.kan.convs.0.conv.grid', 'model.ae_encoder.encoder1.0.weight', 'model.ae_encoder.encoder1.0.bias', 'model.ae_encoder.encoder1.1.weight', 'model.ae_encoder.encoder1.1.bias', 'model.ae_encoder.encoder1.1.running_mean', 'model.ae_encoder.encoder1.1.running_var', 'model.ae_encoder.encoder1.1.num_batches_tracked', 'model.ae_encoder.encoder2.0.weight', 'model.ae_encoder.encoder2.0.bias', 'model.ae_encoder.encoder2.1.weight', 'model.ae_encoder.encoder2.1.bias', 'model.ae_encoder.encoder2.1.running_mean', 'model.ae_encoder.encoder2.1.running_var', 'model.ae_encoder.encoder2.1.num_batches_tracked', 'model.ae_encoder.encoder3.0.weight', 'model.ae_encoder.encoder3.0.bias', 'model.ae_encoder.encoder3.1.weight', 'model.ae_encoder.encoder3.1.bias', 'model.ae_encoder.encoder3.1.running_mean', 'model.ae_encoder.encoder3.1.ru

In [43]:
dft = pd.read_csv('embeddings.csv')

In [44]:
label = pd.read_csv('../../../data/processed/OHEheparfix.csv')['classes']

In [45]:
dftr['label'] = label.values

In [47]:
dftr.to_csv('DAE_Embeddings.csv', index=False)

In [49]:
dftr.he

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,label
0,-0.508204,0.622806,0.963754,1.194291,0.409287,0.537334,1.534004,0.622889,0.647555,1.055628,...,0.155102,-0.088649,-0.526499,-0.947276,0.008985,2.977365,0.273558,0.934428,0.711128,3
1,-0.606001,1.883347,0.765502,0.952280,2.456968,2.221480,1.554836,0.931303,0.939782,1.231403,...,4.324496,2.839384,2.212631,2.402034,4.629514,2.531396,1.919847,2.175018,1.355503,3
2,-0.186743,0.752017,1.203069,0.962292,0.793253,1.775011,2.631131,0.534492,1.263626,0.479104,...,2.658668,2.115646,0.756491,1.330351,2.622540,1.608500,2.752142,4.434757,4.799832,3
3,-0.391148,0.844790,0.851928,1.039518,1.263765,0.471498,0.756313,0.812012,0.356798,0.378331,...,1.729291,3.512805,0.474528,0.291598,0.561205,1.406523,0.627154,-1.026968,-0.592155,3
4,-0.405957,1.519081,0.034942,0.742681,0.784112,0.194307,0.458365,0.387138,-0.251965,-0.117958,...,-0.862648,0.615942,0.299208,0.181120,0.125456,1.014925,0.293138,-1.050816,-1.000228,3


In [22]:
set(train_ds.y.tolist())

{0.0, 1.0}

In [11]:
# dftr['label'] =